# Buliding Flexible Code Base

We will build a flexible code base that increases the ease of development and code-reuse. I will take bits and pieces of FASTAI library and use this as a base for our experiments.

## Let's setup dummy dataset

In [3]:

from sklearn import datasets

noise = 0.1
X_train, y_train = datasets.make_moons(n_samples=2000, noise=noise)

In [4]:
y_train

array([0, 1, 1, ..., 1, 1, 1])

In [5]:
#create dummy dataset
import torch
from torch.utils.data import Dataset,DataLoader
from torch.nn import Sequential
import torch.nn as nn
from tqdm import tqdm
class DummyDset(Dataset):
    def __init__(self,sz,noise=0.1):
        self.x,self.y = datasets.make_moons(n_samples=sz,noise=noise)
        self.x = self.x.astype('float32')
    def __getitem__(self,ind):
        return self.x[ind],self.y[ind]
    def __len__(self):
        return len(self.x)
        
        
class Dloaders:
    def __init__(self,*dls):
        self.train,self.valid = dls

In [6]:
sampl = DummyDset(100)

In [7]:
for x,y in sampl:
    print(f'{x},{x.shape}')
    print(f'{y},{y.shape}')
    break

[ 0.30134398 -0.0563121 ],(2,)
1,()


In [8]:
trn_dset,vld_dset,tst_dset = DummyDset(2000),DummyDset(500),DummyDset(500)

In [9]:
trn_dl = DataLoader(trn_dset,shuffle=True,batch_size=50)
vld_dl = DataLoader(vld_dset,batch_size=50)

In [8]:
dls = Dloaders(trn_dl,vld_dl)

In [9]:
for x_bl,y_bl in trn_dl:
    print(x_bl.shape)
    print(y_bl.shape)
    break

torch.Size([50, 2])
torch.Size([50])


In [14]:
dmodel = Sequential(nn.Linear(2,164),
          nn.LeakyReLU(),
          nn.Linear(164,32),
          nn.LeakyReLU(),
          nn.Linear(32,2),
          )

In [15]:
dmodel

Sequential(
  (0): Linear(in_features=2, out_features=164, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=164, out_features=32, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=32, out_features=2, bias=True)
)

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer_fn = torch.optim.Adam

In [128]:
# define Python user-defined exceptions
class CancelFitException(Exception):
    pass

In [156]:
xb,yb = None,None
for xb,yb in trn_dl:
    break
    

In [157]:
out = dmodel(xb)

## Flexible Trainer Class

In [12]:



class Trainer:
    def __init__(self,model,dls,loss_func,lr,opt_func,cbs,device='cpu'):
        self.model = model
        self.dl = dls
        self.loss_func = loss_func
        self.lr = lr
        self.opt_func = opt_func
        self.cbs =cbs
        self.device = device
        for cb in cbs:cb.trainer = self
        
    def one_batch(self,batch):
        self('before_batch')
        xb,yb = batch
        self.opt.zero_grad() 
        #storing y_preds onto the class for a potential use by a callback
        self.y_preds = self.model(xb)
        self.loss = self.loss_func(self.y_preds,yb)
        if self.model.training:
            #take backward step only if in training mode
            self.loss.backward()
            self.opt.step()
        self('after_batch')
        
    def one_epoch(self,is_train):
        self.model.training = is_train
        self('before_epoch')
        #choose the appropriate data-loader
        dl = self.dl.train if is_train else self.dl.valid
        for self.epochnum,self.batch in enumerate(dl):
            #import pdb;pdb.set_trace()
            self.one_batch(self.batch)
        self('after_epoch')
    
    def fit(self,nepochs):
        self('before_fit')
        self.opt = self.opt_func(self.model.parameters(),self.lr)
        self.nepochs = nepochs
        try:
            for self.epoch in tqdm(range(self.nepochs)):
                self.one_epoch(True)
                self.one_epoch(False)
        except CancelFitException:
            pass
        self('after_fit')
        
    def predict(self,dl):
        print('NOT IMPLEMENTED')
        pass
    
    def infer(self,x):
        pass
    
    def __call__(self,name):
        for cb in self.cbs:getattr(cb,name)()
        
        

In [19]:
class CallBack:
    def before_epoch(self):
        pass
    def after_epoch(self):
        pass
    def before_batch(self):
        pass
    def after_batch(self):
        pass
    def before_fit(self):
        pass
    def after_fit(self): 
        pass
    
    
    

class SetupTrainerCB(CallBack):
    def before_batch(self):
        xb,yb = to_device(self.trainer.device,self.trainer.batch)
        self.trainer.batch = xb,yb

    def before_fit(self): 
        if self.trainer.device != 'cpu':
            self.trainer.model.cuda()

In [20]:
class PrinterCB(CallBack):
    def before_epoch(self):
        self.ns,self.losses,self.accs = [],[],[]
        
    def after_epoch(self):
        n = sum(self.ns)
        epoch_type = 'Training' if self.trainer.model.training else 'Validation'
        acc = sum(self.accs)/n
        loss = sum(self.losses)/n
        print(f"{epoch_type} acc: {acc},loss:{loss}")
        
    def after_batch(self):
        xb,yb = self.trainer.batch
        acc = (self.trainer.y_preds.argmax(dim=-1) == yb).float().sum()
        ns = len(xb)
        l = (self.trainer.loss.item())*ns
        self.ns.append(ns)
        self.losses.append(l)
        self.accs.append(acc)
    

In [21]:
cbs = [PrinterCB()]

In [22]:
trainer = Trainer(dmodel,dls,loss_fn,0.0002,optimizer_fn,cbs,'cpu')

In [23]:
trainer.fit(10)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 41.63it/s]

Training acc: 0.6265000104904175,loss:0.6529418170452118
Validation acc: 0.777999997138977,loss:0.5868911445140839
Training acc: 0.8105000257492065,loss:0.5206522881984711
Validation acc: 0.8119999766349792,loss:0.4602777689695358
Training acc: 0.8289999961853027,loss:0.40951949581503866
Validation acc: 0.8299999833106995,loss:0.3736293613910675
Training acc: 0.8510000109672546,loss:0.3374750755727291
Validation acc: 0.8539999723434448,loss:0.3180138051509857
Training acc: 0.8734999895095825,loss:0.289821594581008
Validation acc: 0.8700000047683716,loss:0.2798048108816147
Training acc: 0.887499988079071,loss:0.25773362182080745
Validation acc: 0.8920000195503235,loss:0.2542142793536186
Training acc: 0.8999999761581421,loss:0.23499636426568032
Validation acc: 0.8980000019073486,loss:0.23408746123313903
Training acc: 0.9079999923706055,loss:0.21637702491134406
Validation acc: 0.8960000276565552,loss:0.21646395623683928
Training acc: 0.9144999980926514,loss:0.1996240969747305
Validation a

## Using tools

In [10]:
import os
import sys
import torch
import numpy as np
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from torch.nn import functional as F

import pytorch_lightning as pl


from pathlib import Path
import matplotlib.pyplot as plt
#import pandas as pd
import numpy as np

cwd = os.getcwd()
module_path = "/".join(cwd.split('/')[0:-1])
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
from src.models import vgg16


def accuracy(y_hat,y):
    #import pdb;pdb.set_trace()
    return torch.mean((y_hat == y).type(torch.float32))

In [12]:
#configuring pytorch-lightning with wandb
import pytorch_lightning as pl

In [16]:
str_model = {'dmodel':dmodel}

hparams = {'model':'dmodel','optimizer':{'lr':0.0002}}

In [17]:
class Model(pl.LightningModule):
    def __init__(self,hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.model = str_model[hparams['model']]
        #import pdb;pdb.set_trace()
    def forward(self,x):
        #? write for reading off any model specific hparams before the call(hydra)
        x = self.model(x)
        return x
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self.model(x)
        #y_true = y.squeeze().type(torch.LongTensor)
        loss = F.cross_entropy(y_hat,y)
        
        acc = torch.mean((y_hat.argmax(dim=-1) == y).type(torch.float32))
        #import pdb;pdb.set_trace()
        #print(f'batch indx:{batch_idx},{loss.item()}')
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        self.log("train_loss", loss,on_step=False, on_epoch=True)
        
        
        return loss
        
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self(x)
        
        #y_hat = torch.argmax(y_hat, dim=1)
        #import pdb;pdb.set_trace()
        acc = torch.mean((y_hat.argmax(dim=-1) == y).type(torch.float32))
        self.log('val_acc', acc,on_step=False, on_epoch=True)
        return acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['optimizer']['lr'])
        return optimizer

        
        

In [18]:

pl.seed_everything(0)
#model
net = Model(hparams)


Global seed set to 0


In [19]:
from pytorch_lightning.loggers import CSVLogger
logger = CSVLogger("logs", name="temp")
trainer = pl.Trainer(max_epochs=10,logger=logger)
trainer.fit(net,trn_dl,vld_dl)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 5.8 K 
-------------------------------------
5.8 K     Trainable params
0         Non-trainable params
5.8 K     Total params
0.023     Total estimated model params size (MB)


/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 0
/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:412: UserWarning: The number of training sam

Epoch 0:  80%|█████████▌  | 40/50 [00:00<00:00, 404.19it/s, loss=0.639, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 1:  80%|█████████▌  | 40/50 [00:00<00:00, 468.66it/s, loss=0.525, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 2:  80%|███████████▏  | 40/50 [00:00<00:00, 489.90it/s, loss=0.4, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 3:  80%|█████████▌  | 40/50 [00:00<00:00, 495.48it/s, loss=0.321, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 4:  80%|█████████▌  | 40/50 [00:00<00:00, 495.85it/s, loss=0.264, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 5:  80%|█████████▌  | 40/50 [00:00<00:00, 492.89it/s, loss=0.225, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 6:  80%|██████████▍  | 40/50 [00:00<00:00, 491.23it/s, loss=0.19, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 7:  80%|█████████▌  | 40/50 [00:00<00:00, 485.77it/s, loss=0.172, v_num=4]
Validating: 0it [00:00, ?it/s]
Epoch 8:  80%|█████████▌  | 40/50 [00:00<00:00, 502.20it/s, loss=0.152, v_num=4]
Validating: 0it [00:00,

In [20]:
import pandas as pd


In [21]:
log_dir = '/Users/vinay/Projects/uncertainty-estimates/nbs/logs/temp/version_4/'
met = pd.read_csv(log_dir+'metrics.csv')

In [22]:
met

,val_acc,epoch,step,train_acc,train_loss
0,0.500,0,39,NaN,NaN
1,NaN,0,39,0.5000,0.667147
2,0.840,1,79,NaN,NaN
3,NaN,1,79,0.6930,0.560631
4,0.862,2,119,NaN,NaN
5,NaN,2,119,0.8540,0.429429
6,0.884,3,159,NaN,NaN
7,NaN,3,159,0.8735,0.328797
8,0.900,4,199,NaN,NaN
9,NaN,4,199,0.8900,0.266220


In [18]:
class Model(pl.LightningModule):
    def __init__(self,hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.model = str_model[hparams['model']]
        #import pdb;pdb.set_trace()
    def forward(self,x):
        #? write for reading off any model specific hparams before the call(hydra)
        x = self.model(x)
        return x
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self.model(x)
        #y_true = y.squeeze().type(torch.LongTensor)
        loss = F.cross_entropy(y_hat,y)
        
        acc = torch.mean((y_hat.argmax(dim=-1) == y).type(torch.float32))
        #import pdb;pdb.set_trace()
        #print(f'batch indx:{batch_idx},{loss.item()}')
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        self.log("train_loss", loss)
        
        
        return loss
        
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_hat = self(x)
        
        #y_hat = torch.argmax(y_hat, dim=1)
        #import pdb;pdb.set_trace()
        acc = torch.mean((y_hat.argmax(dim=-1) == y).type(torch.float32))
        self.log('val_acc', acc,on_step=False, on_epoch=True)
        return acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['optimizer']['lr'])
        return optimizer

        
        

In [1]:
## Integrating wandb


import wandb


from pytorch_lightning.loggers import WandbLogger

wandb.login()



/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
wandb: Currently logged in as: vin136 (use `wandb login --relogin` to force relogin)


True

In [20]:
wandb_logger = WandbLogger(project="temp-tools")
from pytorch_lightning.loggers import CSVLogger
csvlogger = CSVLogger("logs", name="temp")



In [22]:

pl.seed_everything(0)
#model
net = Model(hparams)




trainer = pl.Trainer(
    logger=[wandb_logger,csvlogger],    # W&B integration
    log_every_n_steps=50,   # set the logging frequency                
    max_epochs=10,           # number of epochs
    deterministic=True,     # keep it deterministic
    )

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [23]:
trainer.fit(net,trn_dl,vld_dl)





  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 5.8 K 
-------------------------------------
5.8 K     Trainable params
0         Non-trainable params
5.8 K     Total params
0.023     Total estimated model params size (MB)


/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 0
/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/vinay/mambaforge/envs/uncertainty-estimates/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:412: UserWarning: The number of training sam

Epoch 0:  80%|███████▏ | 40/50 [00:00<00:00, 331.27it/s, loss=0.581, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 1:  80%|███████▏ | 40/50 [00:00<00:00, 403.25it/s, loss=0.453, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 2:  80%|███████▏ | 40/50 [00:00<00:00, 296.53it/s, loss=0.358, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 3:  80%|███████▏ | 40/50 [00:00<00:00, 358.06it/s, loss=0.283, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 4:  80%|███████▏ | 40/50 [00:00<00:00, 445.32it/s, loss=0.262, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 5:  80%|███████▏ | 40/50 [00:00<00:00, 464.28it/s, loss=0.214, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 6:  80%|███████▏ | 40/50 [00:00<00:00, 410.86it/s, loss=0.187, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 7:  80%|███████▏ | 40/50 [00:00<00:00, 301.42it/s, loss=0.159, v_num=8l_5]
Validating: 0it [00:00, ?it/s]
Epoch 8:  80%|███████▏ | 40/50 [00:00<00:00, 407.58it/s, loss=0.163, v_num=8l_5]
Validating: 0it [00:00,

In [25]:
import pandas as pd
log_dir = '/Users/vinay/Projects/uncertainty-estimates/nbs/logs/temp/version_5/'
met = pd.read_csv(log_dir+'metrics.csv')

In [26]:
met

,val_acc,epoch,step,train_acc,train_loss
0,0.848,0,39,NaN,NaN
1,NaN,0,39,0.7755,NaN
2,NaN,1,49,NaN,0.532628
3,0.860,1,79,NaN,NaN
4,NaN,1,79,0.8450,NaN
5,NaN,2,99,NaN,0.354978
6,0.872,2,119,NaN,NaN
7,NaN,2,119,0.8555,NaN
8,NaN,3,149,NaN,0.221781
9,0.886,3,159,NaN,NaN


## wandb sweeps

In [27]:
import wandb

sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "epochs" : {
      "values" : [10, 20, 50]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 8dlplixp
Sweep URL: https://wandb.ai/vin136/uncategorized/sweeps/8dlplixp


In [28]:
wandb.config

{'model': 'dmodel', 'optimizer/lr': 0.0002}

In [35]:
'/'.join(os.getcwd().split('/')[:-1])

'/Users/vinay/Projects/uncertainty-estimates'

In [50]:
from torch import as_tensor,Tensor,ByteTensor,LongTensor,FloatTensor,HalfTensor,DoubleTensor
from types import GeneratorType

def is_listy(x):
    "`isinstance(x, (tuple,list,slice,GeneratorType))`"
    return isinstance(x, (tuple,list,slice,GeneratorType))



def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if is_listy(x): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res 

def to_device(b, device=None, non_blocking=False):
    "Recursively put `b` on `device`."
    #if defaults.use_cuda==False: device='cpu'
    #elif device is None: device=default_device()
    def _inner(o):
        if isinstance(o,Tensor): return o.to(device, non_blocking=non_blocking)
#         if hasattr(o, "to_device"): return o.to_device(device)
        return o
    return apply(_inner, b)